In [1]:
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
import cv2
from scipy.spatial.distance import cosine
import os

In [4]:
dataset_path = Path(r"C:\Users\197as\OneDrive\Documents\FinFiesta\Face_Recognition")
train_dir = dataset_path / "train"
test_dir = dataset_path / "val"

In [5]:
image_res = 160 
transform = transforms.Compose([
    transforms.Resize((image_res, image_res)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [6]:
train_data = datasets.ImageFolder(root=train_dir, transform=transform)
test_data = datasets.ImageFolder(root=test_dir, transform=transform)

In [ ]:
BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()

train_dataloader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f"Loaded Dataset: Train Samples: {len(train_data)}, Test Samples: {len(test_data)}")
print(f"Classes: {train_data.classes}")

Loaded Dataset: Train Samples: 176398, Test Samples: 21295
Classes: ['n000002', 'n000003', 'n000004', 'n000005', 'n000006', 'n000007', 'n000008', 'n000010', 'n000011', 'n000012', 'n000013', 'n000014', 'n000015', 'n000016', 'n000017', 'n000018', 'n000019', 'n000020', 'n000021', 'n000022', 'n000023', 'n000024', 'n000025', 'n000026', 'n000027', 'n000028', 'n000030', 'n000031', 'n000032', 'n000033', 'n000034', 'n000035', 'n000036', 'n000037', 'n000038', 'n000039', 'n000041', 'n000042', 'n000043', 'n000044', 'n000045', 'n000046', 'n000047', 'n000048', 'n000049', 'n000050', 'n000051', 'n000052', 'n000053', 'n000054', 'n000055', 'n000056', 'n000057', 'n000058', 'n000059', 'n000060', 'n000061', 'n000062', 'n000063', 'n000064', 'n000065', 'n000066', 'n000067', 'n000068', 'n000069', 'n000070', 'n000071', 'n000072', 'n000073', 'n000074', 'n000075', 'n000076', 'n000077', 'n000079', 'n000080', 'n000081', 'n000083', 'n000084', 'n000085', 'n000086', 'n000087', 'n000088', 'n000089', 'n000090', 'n00009

# Model

In [ ]:
class FaceRecognitionModel(nn.Module):
    def __init__(self, embedding_dim=512):
        super(FaceRecognitionModel, self).__init__()
        base_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])  
        self.fc = nn.Linear(base_model.fc.in_features, embedding_dim)  
        self.normalize = nn.functional.normalize

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return self.normalize(x, p=2, dim=1)

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = FaceRecognitionModel().to(device)
print("Face Recognition Model Loaded!")

cuda
Face Recognition Model Loaded!


In [16]:
class ArcFaceLoss(nn.Module):
    def __init__(self, scale=30.0, margin=0.5):
        super(ArcFaceLoss, self).__init__()
        self.scale = scale
        self.margin = margin

    def forward(self, logits, labels):
        theta = torch.acos(torch.clamp(logits, -1.0, 1.0))  
        theta_m = theta + self.margin
        logits_m = torch.cos(theta_m)
        logits_m = logits_m * self.scale
        return F.cross_entropy(logits_m, labels)

In [17]:
epochs = 2
learning_rate = 0.001
criterion = ArcFaceLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        embeddings = model(images)
        loss = criterion(embeddings, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_dataloader)}")

print("Training Complete!")

Epoch 1/2, Loss: 2.7749042016136953
Epoch 2/2, Loss: 1.093436038853057
✅ Training Complete!


# Saving model 

In [18]:

model_save_path = r"C:\Users\197as\OneDrive\Documents\FinFiesta\face_recognition_model.pth"

torch.save(model.state_dict(), model_save_path)  
print(f"Model saved successfully at {model_save_path}!")
  
def load_trained_model(model_path):
    model = FaceRecognitionModel().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()  # Set to evaluation mode
    print(" Model Loaded from Disk!")
    return model
  
model = load_trained_model(model_save_path)


Model saved successfully at C:\Users\197as\OneDrive\Documents\FinFiesta\face_recognition_model.pth!


C:\Users\197as\AppData\Local\Temp\ipykernel_20796\3641471914.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=de

 Model Loaded from Disk!
